In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
mean = X_train.mean()
std = X_train.std()

In [6]:
mean, std

(33.318421449829934, 78.56748998339798)

In [7]:
X_train = (X_train.astype('float32') - mean) / std
X_test = (X_test.astype('float32') - mean) / std

#### Building model

In [8]:
class ConvBlock(layers.Layer):
    def __init__(self, out_channels, kernel=3):
        super(ConvBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel, padding='same')
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor, training=False)
        x = self.bn(x, training=training)
        x = keras.activations.relu(x)
        return x

In [9]:
class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = ConvBlock(channels[0])
        self.cnn2 = ConvBlock(channels[1])
        self.cnn3 = ConvBlock(channels[2])
        self.identity_mapping = layers.Conv2D(channels[0], 1, padding='same')
        self.pooling = layers.MaxPooling2D()

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training)
        x = self.pooling(x)
        return x

In [10]:
class Resnet_like(keras.Model):
    def __init__(self, num_classes=10):
        super(Resnet_like, self).__init__()
        self.block1 = ResBlock([32,32,64])
        self.block2 = ResBlock([128,128,256])
        self.block3 = ResBlock([256,256,512])
        self.pool = layers.GlobalAveragePooling2D()
        self.fn = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.fn(x)
        return x

    def model(self):
        x = keras.Input(shape=(28,28,1))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [11]:
model = Resnet_like().model()

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 res_block (ResBlock)        (None, 14, 14, 64)        28640     
                                                                 
 res_block_1 (ResBlock)      (None, 7, 7, 256)         526976    
                                                                 
 res_block_2 (ResBlock)      (None, 3, 3, 512)         2430208   
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                5130      
                                                             

In [13]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

In [14]:
model.fit(
    X_train, y_train,
    batch_size = 32,
    epochs = 3, 
    validation_data=(X_test, y_test)
)

Epoch 1/3
1875/1875 [==============================] - 52s 25ms/step - loss: 0.1308 - accuracy: 0.9587 - val_loss: 0.0571 - val_accuracy: 0.9805
Epoch 2/3
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0459 - accuracy: 0.9864 - val_loss: 0.0331 - val_accuracy: 0.9895
Epoch 3/3
1875/1875 [==============================] - 54s 29ms/step - loss: 0.0330 - accuracy: 0.9896 - val_loss: 0.0447 - val_accuracy: 0.9875
